In [1]:
### Right now I am only interested in Markovian chains with parameterization, thus in this code I have to make a standard for the input 
### of the code, such that it will have the parametarization functionality, the code has to identify State variables 
### and it has to build the subproblems in dicitonaries such that the first stage problem would be seperate from otherr stages. 
### The code should also log a lot of information for debugging purpose 

### First load all the required modules 

In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import sympy as sp  # used for data storage and debugging purposes: 
import numpy as np
import random

### The classes used 

In [3]:
# This class creates a markov chain

class MarkovChain:
    def __init__(self, t_matrix, stages, states):
        self.t_matrix = t_matrix
        self.stages = list(range(1,stages+1))
        self.states = list(range(1, states +1))

    def nodes(self):
        nodes = [(1,1)]
        for i in self.stages[1:]:
            for j in self.states:
                nodes.append((i,j))
        return nodes
    
    def edges(self):
        edges = {}
        for j in self.states:
                edges[(1,1),(2,j)] = t_matrix[1][j-1]
        for i in self.stages[2:]:
            for j in self.states:
                for k in self.states:
                    edges[(i-1,j),(i,k)] = t_matrix[i-1][j-1][k-1]
        return edges


    def sample_path(self):
        all_edges = self.edges()
        path = [(1,1)]
        for i in self.stages[1:]: 
            path.append((i,random.choice(self.states)))
        return path


# This class is meant to give every node children and parent attribute useful later for the algorithm.
class Node:
    def __init__(self, node, stages, states):
        self.node_name = node
        self.states = list(range(1, states +1))
        
        if node[0] == 1:
            self.parents  = None
            self.children = [(2,s) for s in self.states]
        elif node[0] == 2:
            self.parents = [(1,1)]
            self.children = [(3,s) for s in self.states]
        elif node[0] > 2 &  node[0] < stages:
            self.parents = [(node[0]-1,s) for s in self.states]
            self.children = [(node[0]+1,s) for s in self.states]
        elif node[0] == stages:
            self.parents  = [(node[0]-1,s) for s in self.states]
            self.children = None


# This class is meant to define the nodes on every stage, useful later for the algorithm. 
class Stage:
    def __init__(self, stages, states):
        self.stages = list(range(1,stages+1))
        self.states = list(range(1, states +1))
        self.nodes = {}
        
        for i in self.stages:
            if i == 1:
                self.nodes[i] = [(1,1)]
            else:
                self.nodes[i] = [(i,j) for j in self.states]


### Define all of the subproblem parameters minding the stage and state parameters. 

In [4]:
S = np.matrix([[0,1,2],[0,0,1],[0,0,0]])

t = [60,60,245]

D = [210, 210, 858]

q = [[[4.5,4.5,4.5],[4.5,4.5,4.5],[4.5,4.5,4.5]],
     [[5.5,5.5,5.5],[5.5,5.5,5.5],[5.5,5.5,5.5]],
     [[6.5,6.5,6.5],[6.5,6.5,6.5],[6.5,6.5,6.5]]]

b = [[30,  75,    37.5],
     [15,  37.5,  18.25],
     [7.5, 18.75, 9.325]]

w = 3000  

C = [[50, 50, 50],
     [50, 50, 50],
     [50, 50, 50]] 

r = [[[5, 5, 5], [5, 5, 5], [5, 5, 5]],
     [[6, 6, 6], [6, 6, 6], [6, 6, 6]],
     [[7, 7, 7], [7, 7, 7], [7, 7, 7]]]

M = 60.0  
H = 0.0  
V = [0.05, 0.05, 0.05] 
L = 3000.0  


# The T

t_matrix= [1,
 [0.14, 0.69, 0.17],
 [[0.14, 0.69, 0.17], [0.14, 0.69, 0.17], [0.14, 0.69, 0.17]],
 [[0.14, 0.69, 0.17], [0.14, 0.69, 0.17], [0.14, 0.69, 0.17]]]

stages = 4
states = 3


# The model parameters here must be exactly the same name as the parameter itself in the model 
# Indexed parameters must have a nested dictionary with keys that indicate the index of the paremter, for example, if we have volume_x[1], [2] and [3]. then 
# in the dictionary we will have {"volume:{1:0, 2:0, 3:0}} 

init_state = {"acres_x": M, "bales_x": {1:H,2:0,3:0}}


# Just have to add Non in here now for noise and noise probabilities see if this works
# And then accomidate for if there is actually noise. 


# Create the structure of the markovian chain

In [5]:
# Creating the tree structure using the inputs
mc = MarkovChain(t_matrix, stages, states)  

# Creating a dictionary containing node info such as children and parents 
Nodes = {}
for i in mc.nodes():                 
    Nodes[i] = Node(i,stages,states)

#Defining the probabilities of the edges for cut calculation 
Pij = mc.edges()

#A dictionary containing Stages in each node
Stages = Stage(stages,states)


### Define the Subproblem in the subproblem builder

#### There are a few crucial steps when defining the subprobem the most crucial are defining attributes to state variables, dummy parameters and dummy constraints, and those have to take a very specific form and have homogeniety in their naming. Lets assume that we have a state variable named "volume". Then the following needs to be done: 

 ##### 1- define the state (outgoing) variable as m.volume = Var() 
 ##### 2- give the variable an extra attribute by adding m.volume.state_variable = True
 ##### 3- define the dummy parameter with an added "_x"  as in  m.volume_x = Param()
 ##### 4- give the dummy parameter an extra attribute by adding m.volume_x.dummy = True
 ##### 5- define the dummy constraint, the dummy constraint must have the follwoing naming convention : m.con_\<Variable Name\>x = Constraint(), in this example, that would be m.con_volumex = Constraint. 
 ##### 6- give the dummy constraint an extra attribute by adding m.con_volumex.dummy_constraint = True

##### Whether the constraints or variables have indecies doesn't matter, the code already accomidates for that. 

#### if the above is not followed, the generalization fails, and the algoirthm fails

In [6]:
def subproblem_builder():
    nodes = mc.nodes()
    subproblems = {}
    for i in nodes:
            stage = i[0]
            state = i[1]
            print("subproblem", i, "stage", stage, "state", state)

            m = ConcreteModel()
            
            # Identify the state variables 
        
            m.acres = Var(within = NonNegativeReals, bounds = (0,M))
            m.acres_in = Var(within =NonNegativeReals, bounds = (0,M))
            m.bales = Var([1,2,3], within=NonNegativeReals)
            m.bales_in = Var([1,2,3], within=NonNegativeReals)

            m.acres.state_variable = True
            m.bales.state_variable = True


            # Identify the dummy constraint parameters:
            
            m.acres_x = Param(initialize=0, mutable=True)
            m.bales_x = Param([1,2,3], initialize=0, mutable=True)

            m.acres_x.dummy = True
            m.bales_x.dummy = True
            
            #Local Variables
            
            m.cost_to_go = Var(within=NonNegativeReals, initialize=0)
            

            
            if stage == 1:
            
                m.obj  = Objective(expr = 0.0000001*m.acres + m.cost_to_go, sense= minimize)
                m.con1 = Constraint(expr = m.acres <= m.acres_in)
            
                def con2(m,b):
                    return m.bales[b] == m.bales_in[b]
                m.con2 = Constraint([1,2,3], rule = con2)
            
                m.con_acresx = Constraint(expr = m.acres_in == m.acres_x)
            
                def con_balesx(m,b):
                    return m.bales_in[b] == m.bales_x[b]
                m.con_balesx = Constraint([1,2,3], rule = con_balesx )
            
                m.cuts = ConstraintList()

                #define dummy_constraints
                
                m.con_acresx.dummy_constraint = True
                m.con_balesx.dummy_constraint = True

            
            else:

                m.buy = Var([1,2,3], within=NonNegativeReals)
                m.sell = Var([1,2,3], within=NonNegativeReals)
                m.eat = Var([1,2,3], within=NonNegativeReals)
                m.pen_p = Var([1,2,3], within=NonNegativeReals)
                m.pen_n = Var([1,2,3], within=NonNegativeReals)
            
            
                m.obj  = Objective(expr = 1000 * (sum(m.pen_p[c] for c in [1,2,3]) + sum(m.pen_n[c] for c in [1,2,3])) + C[stage-2][state-1] * m.acres_in +
                sum( V[stage-2] * m.bales_in[c] * t[stage-2] +\
                     r[c-1][stage-2][state-1] * m.buy[c] +\
                     S[c-1, stage-2] * m.eat[c] -\
                     q[c-1][stage-2][state-1] * m.sell[c] for c in [1,2,3]) + m.cost_to_go) 
            
                m.con3 = Constraint(expr = m.acres <= m.acres_in)
            
                m.con4 = Constraint(expr = sum(m.eat[c] for c in [1,2,3]) >= D[stage-2])
            
                def cutex(m,b):
                    return m.bales_in[b] + m.buy[b] - m.eat[b] - m.sell[b] + m.pen_p[b] - m.pen_n[b]
                m.cutex = Expression([1,2,3], rule = cutex)
            
                m.con5 = Constraint(expr = m.bales[stage-1] == m.cutex[stage-1] + m.acres_in * b[stage-2][state-1])
                                    
                def con6(m,b):
                    return m.bales[b] == m.cutex[b]
                m.con6 = Constraint({1,2,3} - {stage-1}, rule = con6)
            
                m.con7 = Constraint(expr = sum(m.bales[b] for b in [1,2,3]) <= w)
            
                def con8(m,b):
                    return m.sell[b] <= m.bales_in[b]
                m.con8 = Constraint({1,2,3}, rule = con8)
            
                m.con9 = Constraint(expr = sum(m.sell[b] for b in [1,2,3]) <= L)
            
                m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
                
                m.con_acresx = Constraint(expr = m.acres_in == m.acres_x)
            
                def con_balesx(m,b):
                    return m.bales_in[b] == m.bales_x[b]
                m.con_balesx = Constraint([1,2,3], rule = con_balesx )
            
                m.cuts = ConstraintList()

                m.con_acresx.dummy_constraint = True
                m.con_balesx.dummy_constraint = True

            subproblems[i] = m
    return subproblems

sb = subproblem_builder()
opt = SolverFactory('gurobi')


subproblem (1, 1) stage 1 state 1
subproblem (2, 1) stage 2 state 1
subproblem (2, 2) stage 2 state 2
subproblem (2, 3) stage 2 state 3
subproblem (3, 1) stage 3 state 1
subproblem (3, 2) stage 3 state 2
subproblem (3, 3) stage 3 state 3
subproblem (4, 1) stage 4 state 1
subproblem (4, 2) stage 4 state 2
subproblem (4, 3) stage 4 state 3


In [7]:
fpi = {}
bpi = {}  #Currently Unused but should be used

# The following code should be turned into functions. namely, the forward and backward passes. r

for iter in range(20):

    print("Forward pass", iter)
    
    iteration_path = mc.sample_path()

    for step, node in enumerate(iteration_path):
    
        print("\nnode", node)
    
        if node == (1, 1):
        
            for v in sb[1,1].component_objects(Param,active=True):
                 if hasattr(v, 'dummy') == True:
                    for index in v: 
                        if index == None:
                            setattr(sb[1,1], str(v), init_state[str(v)]) 
                        else:
                            setattr(sb[1,1], str(v[index]), init_state[str(v)][index]) 
        
        else:
            
            for v in sb[node].component_objects(Param,active=True):
                 if hasattr(v, 'dummy') == True:
                    for index in v: 
                        if index == None:
                            setattr(sb[node], str(v), fpi[iter,step-1]['var_out'][(0,str(v)[:-2])]) 
                        else:
                            setattr(sb[node], str(v[index]), fpi[iter,step-1]['var_out'][(index, str(v)[:-2])]) 
                            
            # sb[node].acres_x    = fpi[iter,step-1]['var_out'][(0,'acres')]
            # for i in sb[node].bales_x:
            #     sb[node].bales_x[i] = fpi[iter,step-1]['var_out'][(i,'bales')]
                
        results = opt.solve(sb[node])
    
        out_dict = {}
        
        for v in sb[node].component_objects(Var,active=True):
             if hasattr(v, 'state_variable') == True:
                for index in v: 
                    if index == None:
                        out_dict[0, v.name] = value(v[index])
                    else:
                        out_dict[index, v.name] = value(v[index])
    
        fpi[iter,step] = {'node'  : node,
                     'var_out': out_dict,
                     'obj_val': value(sb[node].obj),
                     'cos2go': sb[node].cost_to_go.value,
                     'stg_obj':  value(sb[node].obj) - sb[node].cost_to_go.value,
                     'noise': "None"
                     }
    
        print("Cost to Go (Bellman Term):", fpi[iter,step]['cos2go'])
        print("Objective Value:",fpi[iter,step]['obj_val'])
        print("Stage Objective:", fpi[iter,step]['stg_obj'])


#### -----------------------------##### -------------------------------- ##### 

    print("\nbackward pass")
    
    β = {}
    α = {}
    
    V = {}
    dxdV = {}
    
    # -----------------stage 4 cuts -------------#

    for stage in reversed(Stages.stages):
        print(stage)

        if stage > 1 :
            print('not 1')
            
            for node in Stages.nodes[stage]:
                print(node)

                for v in sb[node].component_objects(Param,active=True):
                     if hasattr(v, 'dummy') == True:
                        for index in v: 
                            if index == None:
                                v.value = fpi[iter,stage-2]['var_out'][(0,'acres')]
                            else:
                                v[index].value = fpi[iter,stage-2]['var_out'][(index, 'bales')]

                if stage == Stages.stages[-1]:
                    sb[node].cost_to_go.fix(0)
            
            
                results = opt.solve(sb[node])
                print("solved ", node)
                V[node] = value(sb[node].obj)
                
                for v in sb[node].component_objects(Constraint,active=True):
                     if hasattr(v, 'dummy_constraint') == True:
                        for index in v: 
                                dxdV[node,str(v)[4:-1],index] = sb[node].dual[v[index]]
            
                for i in Nodes[node].parents:

                    for v in sb[node].component_objects(Var,active=True):
                        if hasattr(v, 'state_variable') == True:
                            for index in v:
                                β[i,node, str(v), index]  = Pij[i,node]*dxdV[node, str(v), index]


                    α[i,node] = Pij[i,node]*V[node]\
                                 - sum(β[i,node,v.name[:-2],index]*v[index].value\
                                           for v in sb[node].component_objects(Param,active=True)\
                                           if hasattr(v, 'dummy') == True\
                                           for index in v)


    
            for i in Stages.nodes[stage-1]:
                sb[i].cuts.add(expr = sb[i].cost_to_go >= sum(α[i,j] for j in Stages.nodes[stage])\
                                       + sum(sum(β[i,j,v.name,index]*v[index] for j in Stages.nodes[stage])\
                                           for v in sb[i].component_objects(Var,active=True)\
                                           if hasattr(v, 'state_variable') == True for index in v))
        
    results = opt.solve(sb[1,1])
    V = value(sb[1,1].obj)
    lower_bound = V
    print(lower_bound)

Forward pass 0

node (1, 1)
Cost to Go (Bellman Term): 0.0
Objective Value: 0.0
Stage Objective: 0.0

node (2, 1)
Cost to Go (Bellman Term): 0.0
Objective Value: 1050.0
Stage Objective: 1050.0

node (3, 2)
Cost to Go (Bellman Term): 0.0
Objective Value: 1260.0
Stage Objective: 1260.0

node (4, 2)
Cost to Go (Bellman Term): 0.0
Objective Value: 6006.0
Stage Objective: 6006.0

backward pass
4
not 1
(4, 1)
solved  (4, 1)
(4, 2)
solved  (4, 2)
(4, 3)
solved  (4, 3)
3
not 1
(3, 1)
solved  (3, 1)
(3, 2)
solved  (3, 2)
(3, 3)
solved  (3, 3)
2
not 1
(2, 1)
solved  (2, 1)
(2, 2)
solved  (2, 2)
(2, 3)
solved  (2, 3)
1
1.8192985440251763e-06
Forward pass 1

node (1, 1)
Cost to Go (Bellman Term): 0.0
Objective Value: 1.8192985440251763e-06
Stage Objective: 1.8192985440251763e-06

node (2, 3)
Cost to Go (Bellman Term): 2765.2659077869844
Objective Value: 3674.915179799573
Stage Objective: 909.6492720125884

node (3, 3)
Cost to Go (Bellman Term): 4932.445573909824
Objective Value: 5842.094845922412


In [8]:
# Forward simulation here is not generalized, Forward simulation should be turned into a function that 
# Calls the forward pass instead of making it its entire own thing. 

fsi = {}
def forward_simulation(replications, fsi):
    rep_results = {}
    for rep in range(replications):
    
        # print("Simulation ", rep)
        
        simulation_path = mc.sample_path()

        for step, node in enumerate(simulation_path):
        
            # print("\nnode", node)
        
            if node == (1, 1):
            
                sb[1,1].acres_x    = init_state['acres_x']
                for i in sb[node].bales_x:
                    sb[1,1].bales_x[i] = init_state['bales_x'][i]
    
                # print( "node 1 input value:", sb[node].acres_x.value )
                
            else:
                
                sb[node].acres_x    = fpi[iter,step-1]['var_out'][(0,'acres')]
                for i in sb[node].bales_x:
                    sb[node].bales_x[i] = fpi[iter,step-1]['var_out'][(i,'bales')]
                    
            results = opt.solve(sb[node])
        
            out_dict = {}
            
            for v in sb[node].component_objects(Var,active=True):
                 if hasattr(v, 'state_variable') == True:
                    for index in v: 
                        if index == None:
                            out_dict[0, v.name] = value(v[index])
                        else:
                            out_dict[index, v.name] = value(v[index])
        
            fsi[rep,node] = {'node'  : node,
                         'var_out': out_dict,
                         'obj_val': value(sb[node].obj),
                         'cos2go': sb[node].cost_to_go.value,
                         'stg_obj':  value(sb[node].obj) - sb[node].cost_to_go.value,
                         'noise': "None"
                         }
        
            # print("Cost to Go (Bellman Term):", fpi[iter,step]['cos2go'])
            # print("Objective Value:",fpi[iter,step]['obj_val'])
            # print("Stage Objective:", fpi[iter,step]['stg_obj'])

        rep_results[rep] = sum(fsi[rep,node]['stg_obj'] for node in simulation_path)
        z_list = list(rep_results.values())
        

    print("| Simulation CI: ", np.mean(z_list), u"\u00B1", (np.std(z_list)*1.96) / np.sqrt(replications))

In [9]:
forward_simulation(20,fsi)

| Simulation CI:  4503.819004280001 ± 637.63561575456


#### The code below is for Debugging purposes, it allows printing the cuts of the subproblems in a simplified form for easier comparision with the SDDP.jl and or non-generalized code comparision.  

In [9]:

# Define the conversion function
def pyomo_to_sympy(expr):
    """
    Convert a Pyomo expression to a SymPy expression, handling inequalities, arithmetic operations, 
    and specific Pyomo expression types including MonomialTermExpression.
    """
    # Check for Python numeric types first
    if isinstance(expr, (int, float)):
        return sp.Float(expr)
    elif expr.is_constant():
        # Convert Pyomo constants
        return sp.Float(value(expr))
    elif expr.is_variable_type():
        # Convert Pyomo variables to SymPy symbols
        return sp.Symbol(expr.name)
    elif expr.__class__.__name__ == 'InequalityExpression':
        # Handle inequality expressions (<=, >=)
        lhs = pyomo_to_sympy(expr.args[0])  # Left-hand side
        rhs = pyomo_to_sympy(expr.args[1])  # Right-hand side
        return lhs <= rhs  # Assuming it's a <= inequality
    elif expr.__class__.__name__ == 'SumExpression':
        # Handle sum expressions
        return sp.Add(*[pyomo_to_sympy(arg) for arg in expr.args])
    elif expr.__class__.__name__ == 'ProductExpression':
        # Handle product expressions
        return sp.Mul(*[pyomo_to_sympy(arg) for arg in expr.args])
    elif expr.__class__.__name__ == 'DivisionExpression':
        # Handle division expressions
        return pyomo_to_sympy(expr.args[0]) / pyomo_to_sympy(expr.args[1])
    elif expr.__class__.__name__ == 'PowerExpression':
        # Handle power expressions
        return sp.Pow(pyomo_to_sympy(expr.args[0]), pyomo_to_sympy(expr.args[1]))
    elif expr.__class__.__name__ == 'NegationExpression':
        # Handle negation expressions
        return -pyomo_to_sympy(expr.args[0])
    elif expr.__class__.__name__ == 'MonomialTermExpression':
        # Handle monomial terms (e.g., c * x)
        return sp.Mul(*[pyomo_to_sympy(arg) for arg in expr.args])
    else:
        raise ValueError(f"Unsupported expression type: {expr.__class__.__name__}")


# Convert the Pyomo expression to a SymPy expression


In [17]:
s = 1
n = 1

for i in range(1,6):
    sympy_expr = pyomo_to_sympy(sb[s,n].cuts[i].expr)
    # print("SymPy Expression:", sympy_expr)
    simplified_expr = sp.simplify(sympy_expr)
    print(simplified_expr)

457.09925*acres + 2.0*bales[1] + 3.0*bales[2] + 4.0*bales[3] + 1.0*cost_to_go >= 8316.0
52.49675*acres + 1.5*bales[1] + 3.0*bales[2] + 4.0*bales[3] + 1.0*cost_to_go >= 6321.0
-35.2*acres + 1.5*bales[1] + 3.0*bales[2] + 4.0*bales[3] + 1.0*cost_to_go >= 2518.5938726249
-25.12825*acres + 1.5*bales[1] + 3.0*bales[2] + 4.0*bales[3] + 1.0*cost_to_go >= 2998.65
-25.12825*acres + 1.5*bales[1] + 3.0*bales[2] + 4.0*bales[3] + 1.0*cost_to_go >= 2998.65


### Next Step  


next step: Next step is to try and replicate a simple problem with binary and continues state and local variables in both monolithically and using the algorithm: make it very simple like minimal noise and minimal. 

future: Make a Generalized Code for solving the problem with Noise 

After that works, I can try and model my actual model full model from the paper using python

Once all of that is done, I can then try step by step different parallel algorithms on any of the small problems that contain both long and short term uncertainity. 

Finally I can go ahead and attempt that on my large scale model. 

Once all of that is done, I can then Experiment and Look into different types of cuts to make, and also different convergence Criteria. 

Finally, 



## Imporvement notes

#### The code is clearly still very messy and will require further enhancement and data logging

#### Performance enhancement:
##### 1-  Must figure out a way to remove redaundant cuts from subproblems, i.e. cuts that are repeated and cuts that are useless and have no effect on the solution 
